In [1]:
import numpy as np
import pandas as pd
import time
%load_ext line_profiler

In [2]:
data = pd.read_csv("adult.data", delimiter=", ", engine='python', names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex',
                                                                         'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'])

In [3]:
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [4]:
temp = data[["age", "fnlwgt", "education-num", "income"]].copy()
temp["income"] = (temp.income==">50K").astype('int')
y = temp[["income"]].values
X = temp[["age", "fnlwgt", "education-num"]].copy().to_numpy()

## Algorithm

In [16]:
def train(X, y, bs, epochs, lr):
    m2, n2 = X.shape
    
    # Initializing weights and bias to zeros.
    w = np.zeros((n2,1))
    b = 0
    
    # Reshape y.
    y = y.reshape(m2,1)
    
    # Normalize inputs
    #x = normalize(X)
    #____________________change 1
    m, n = X.shape
    for i in range(n):
        X = (X - X.mean(axis=0))/X.std(axis=0)
    #______________________________
    
    
    # Store losses
    losses = []
    
    # Train
    for epoch in range(epochs):
        for i in range((m2-1)//bs + 1):
            
            # Defining batches for SGD (this can be changed)
            start_i = i*bs
            end_i = start_i + bs
            xb = X[start_i:end_i]
            yb = y[start_i:end_i]
            
            # Predict
            #y_hat = sigmoid(np.dot(xb, w) + b)
            #____________________change 4
            y_hat =1.0/(1 + np.exp(-(np.dot(xb, w) + b)))
            #_____________________
            
            
            # Calculate gradients
            #dw, db = gradients(xb, yb, y_hat)
            #____________________________change 2
            m1 = xb.shape[0]
            # Gradient of loss w.r.t weights
            #dw = (1/m1)*np.dot(xb.T,(y_hat - yb))
            # Gradient of loss w.r.t bias
            #db = (1/m1)*np.sum((y_hat - yb)) 
            #______________________________
            # Update params
            w -= lr*(1/m1)*np.dot(xb.T,(y_hat - yb))
            b -= lr*(1/m1)*np.sum((y_hat - yb)) 
        
        # Calc loss
        #l = loss(x, y, w)
        #_________________________change 3
        margin = np.dot(X, w)
        losses.append(y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin)))))
        #_________________________
    return w, b, losses

In [17]:
def predict(X):
    
    # X --> Input.
    
    # Normalizing the inputs.
    #x = normalize(X)
    #_____________________change 6
    m, n = X.shape
    for i in range(n):
        X = (X - X.mean(axis=0))/X.std(axis=0)
    #_______________________
    
    # Calculating presictions/y_hat.
    #preds = sigmoid(np.dot(X, w) + b)
    #______________________change 5
    preds = 1.0/(1 + np.exp(-(np.dot(X, w) + b)))
    #_______________________
    
    # if y_hat >= 0.5 --> round up to 1
    # if y_hat < 0.5 --> round up to 1
    pred_class = [1 if i > 0.5 else 0 for i in preds]
    
    return np.array(pred_class)

In [18]:
def accuracy(y, y_hat):
    accuracy = np.sum(y == y_hat) / len(y)
    return accuracy

In [19]:
%lprun -f train w, b, l = train(X, y, bs=100, epochs=1000, lr=0.001)

In [12]:
%%time
# Training 
for i in range(10):
    w, b, l = train(X, y, bs=100, epochs=1000, lr=0.001)

Wall time: 3min 57s


In [13]:
print(f'weights: {w}, beta: {b}')

weights: [[0.59660644]
 [0.04652271]
 [0.93286765]], beta: -1.4200837383939489


In [14]:
from logistic_regression import LogisticRegression

ModuleNotFoundError: No module named 'logistic_regression'

In [ ]:
lr = LogisticRegression(max_epochs=1000, learning_rate=0.001, batch_size=100)
lr.train(X, y)

In [ ]:
print(f'weights: {lr.get_weights()}, beta: {lr.get_bias()}')

## TO DO:

### Step 1: Clean (and slow) baseline

- Change code into singular class? (for presentability)

- Implement minibatch if it isn't already

- Add in regularization

- Replace np.sum(), np.dot(), np.mean(), etc. with handmade functions

- Implement data science pipeline (including grid search)


### Step 2: Speed up

Find different methods to improve baseline and implement (look through syllabus):

1) Use numpy

2) Cython

3) Parallel computing (esp. for gradient descent)

4) ...


#### Minor step 3: Get clean output


Timer unit: 1e-07 s

Total time: 34.0118 s
File: <ipython-input-9-7301afd39c54>
Function: train at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def train(X, y, bs, epochs, lr):
     2         1         64.0     64.0      0.0      m2, n2 = X.shape
     3                                               
     4                                               # Initializing weights and bias to zeros.
     5         1        120.0    120.0      0.0      w = np.zeros((n2,1))
     6         1         11.0     11.0      0.0      b = 0
     7                                               
     8                                               # Reshape y.
     9         1         96.0     96.0      0.0      y = y.reshape(m2,1)
    10                                               
    11                                               # Normalize inputs
    12                                               #x = normalize(X)
    13                                               #____________________change 1
    14         1         13.0     13.0      0.0      m, n = X.shape
    15         4        779.0    194.8      0.0      for i in range(n):
    16         3      66893.0  22297.7      0.0          X = (X - X.mean(axis=0))/X.std(axis=0)
    17                                               #______________________________
    18                                               
    19                                               
    20                                               # Store losses
    21         1         12.0     12.0      0.0      losses = []
    22                                               
    23                                               # Train
    24      1001      29475.0     29.4      0.0      for epoch in range(epochs):
    25    327000    4837793.0     14.8      1.4          for i in range((m2-1)//bs + 1):
    26                                                       
    27                                                       # Defining batches for SGD (this can be changed)
    28    326000    4746645.0     14.6      1.4              start_i = i*bs
    29    326000    4810686.0     14.8      1.4              end_i = start_i + bs
    30    326000    7123366.0     21.9      2.1              xb = X[start_i:end_i]
    31    326000    6585917.0     20.2      1.9              yb = y[start_i:end_i]
    32                                                       
    33                                                       # Predict
    34                                                       #y_hat = sigmoid(np.dot(xb, w) + b)
    35                                                       #____________________change 4
    36    326000   84449182.0    259.0     24.8              y_hat =1.0/(1 + np.exp(-(np.dot(xb, w) + b)))
    37                                                       #_____________________
    38                                                       
    39                                                       
    40                                                       # Calculate gradients
    41                                                       #dw, db = gradients(xb, yb, y_hat)
    42                                                       #____________________________change 2
    43    326000    6527809.0     20.0      1.9              m1 = xb.shape[0]
    44                                                       # Gradient of loss w.r.t weights
    45                                                       #dw = (1/m1)*np.dot(xb.T,(y_hat - yb))
    46                                                       # Gradient of loss w.r.t bias
    47                                                       #db = (1/m1)*np.sum((y_hat - yb)) 
    48                                                       #______________________________
    49                                                       # Update params
    50    326000   67845472.0    208.1     19.9              w -= lr*(1/m1)*np.dot(xb.T,(y_hat - yb))
    51    326000   91315407.0    280.1     26.8              b -= lr*(1/m1)*np.sum((y_hat - yb)) 
    52                                                   
    53                                                   # Calc loss
    54                                                   #l = loss(x, y, w)
    55                                                   #_________________________change 3
    56      1000     873189.0    873.2      0.3          margin = np.dot(X, w)
    57      1000   60904927.0  60904.9     17.9          losses.append(y * -np.logaddexp(0, np.exp(margin) + (1 - y) * (1 + np.logaddexp(0, np.exp(margin)))))
    58                                                   #_________________________
    59         1         18.0     18.0      0.0      return w, b, losses